In [1]:
import pandas as pd
from data_manager import Document
import pickle
from bs4 import BeautifulSoup

# Dataset

Gotowy dataset wymagający lekkiego preprocessingu został pobrany ze strony kaggle.com.
Jest to zbiór stron z pytaniami i odpowiedziami ze stack overflow z tagiem python.
Pytania pochodzą z lat 2008 - 2016. Początkowy zbiór zawierał ponad 600 tysięcy pytań. Ostatecznie został ograniczony do
około 180 tysięcy.


In [2]:
df = pd.read_csv("data/Questions.csv", encoding="latin-1")
df2 = pd.read_csv("data/Answers.csv", encoding="latin-1")
df3 = pd.read_csv("data/Tags.csv", encoding="latin-1")

In [3]:
answers = {}
for index, row in df2.iterrows():
    p_id = row['ParentId']
    if p_id not in answers:
        answers[p_id] = [row]
    else:
        answers[p_id].append(row)

In [4]:
tags = {}
for index, row in df3.iterrows():
    p_id = row['Id']
    if p_id not in tags:
        tags[p_id] = [row]
    else:
        tags[p_id].append(row)

In [5]:
docs: list[Document] = []

for index, row in df.iterrows():
    p_id = row['Id']
    soup = BeautifulSoup(row['Body'], features='lxml')
    txt = row['Title'] + " " + soup.text

    answer_text = ""
    if p_id in answers:
        answer_text = "\n".join([ans['Body'] for ans in answers[row['Id']]])
        soup = BeautifulSoup(answer_text, features='lxml')
        answer_text = soup.text + " "

    if p_id in tags:
        answer_text += "\n".join([str(tag['Tag']) for tag in tags[row['Id']]])

    txt += answer_text
    docs.append(Document(row['Title'], row['Id'], txt))


In [6]:
with open("data/RawDocuments.pickle", "wb") as f:
    for doc in docs:
        pickle.dump(doc, f)